##Importing the Libraries

In [ ]:
import cv2
import glob
import warnings
import scipy.misc
import numpy as np
import nibabel as nib
!pip install simpleitk
import SimpleITK as sitk
from scipy import ndimage
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, Input

from google.colab import drive
drive._mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Defining the Parameters

In [ ]:
# Image Parameters
IMAGE_SIZE = (256, 128, 256)

# Training, Testing and Validation Parameters
TRAINING_VOLUMES = [0, 1, 2, 3, 4, 5, 6, 7, 8]
VALIDATION_VOLUMES = [9]

# Hyperparameters
N_CLASSES = 4
N_INPUT_CHANNELS = 1
PATCH_SIZE = (32, 32)
PATCH_STRIDE = (32, 32)

# Data Preparation Parameters
CONTENT_THRESHOLD = 0.3 # To Get Rid of Useless Information in the Image

# Training Parameters
N_EPOCHS = 200
BATCH_SIZE = 64
PATIENCE = 20
MODEL_FNAME_PATTERN = 'model.h5'
OPTIMISER = 'Adam'
LOSS = 'categorical_crossentropy'
dropout_rate = 0.40

##Define UNet Architecture

In [ ]:
def get_unet(img_size=PATCH_SIZE, n_classes=N_CLASSES, n_input_channels=N_INPUT_CHANNELS, scale=1):
    inputs = keras.Input(shape=img_size + (n_input_channels, ))

    # Encoding Path of the DenseUNet (32-64-128-256-512)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conc11 = concatenate([inputs, conv11], axis=3)
    conv12 = Conv2D(32, (3, 3), activation='relu', padding='same')(conc11)
    conc12 = concatenate([inputs, conv12], axis=3)
    drop1 = Dropout(rate=dropout_rate)(conc12, training=True)
    pool1 = MaxPooling2D(pool_size=(2, 2))(drop1)

    conv21 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conc21 = concatenate([pool1, conv21], axis=3)
    conv22 = Conv2D(64, (3, 3), activation='relu', padding='same')(conc21)
    conc22 = concatenate([pool1, conv22], axis=3)
    drop2 = Dropout(rate=dropout_rate)(conc22, training=True)
    pool2 = MaxPooling2D(pool_size=(2, 2))(drop2)

    conv31 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conc31 = concatenate([pool2, conv31], axis=3)
    conv32 = Conv2D(128, (3, 3), activation='relu', padding='same')(conc31)
    conc32 = concatenate([pool2, conv32], axis=3)
    drop3 = Dropout(rate=dropout_rate)(conc32, training=True)
    pool3 = MaxPooling2D(pool_size=(2, 2))(drop3)

    conv41 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conc41 = concatenate([pool3, conv41], axis=3)
    conv42 = Conv2D(256, (3, 3), activation='relu', padding='same')(conc41)
    conc42 = concatenate([pool3, conv42], axis=3)
    drop4 = Dropout(rate=dropout_rate)(conc42, training=True)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv51 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conc51 = concatenate([pool4, conv51], axis=3)
    conv52 = Conv2D(512, (3, 3), activation='relu', padding='same')(conc51)
    conc52 = concatenate([pool4, conv52], axis=3)
    drop5 = Dropout(rate=dropout_rate)(conc52, training=True)

    # Decoding Path of the ResUNet
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(drop5), conc42], axis=3)
    conv61 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conc61 = concatenate([up6, conv61], axis=3)
    conv62 = Conv2D(256, (3, 3), activation='relu', padding='same')(conc61)
    conc62 = concatenate([up6, conv62], axis=3)
    drop6 = Dropout(rate=dropout_rate)(conc62, training=True)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(drop6), conv32], axis=3)
    conv71 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conc71 = concatenate([up7, conv71], axis=3)
    conv72 = Conv2D(128, (3, 3), activation='relu', padding='same')(conc71)
    conc72 = concatenate([up7, conv72], axis=3)
    drop7 = Dropout(rate=dropout_rate)(conc72, training=True)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(drop7), conv22], axis=3)
    conv81 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conc81 = concatenate([up8, conv81], axis=3)
    conv82 = Conv2D(64, (3, 3), activation='relu', padding='same')(conc81)
    conc82 = concatenate([up8, conv82], axis=3)
    drop8 = Dropout(rate=dropout_rate)(conc82, training=True)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(drop8), conv12], axis=3)
    conv91 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conc91 = concatenate([up9, conv91], axis=3)
    conv92 = Conv2D(32, (3, 3), activation='relu', padding='same')(conc91)
    conc92 = concatenate([up9, conv92], axis=3)
    drop9 = Dropout(rate=dropout_rate)(conc92, training=True)

    outputs = Conv2D(n_classes, (1, 1), activation='sigmoid')(drop9)

    model = Model(inputs, outputs)

    return model

##Loading the training and validation data

In [ ]:
def load_data_bias(image_size, setName) :
  
  data_file = '/content/drive/My Drive/MISA/Normal Segmentations/data/{}/*'.format(setName)

  folders = glob.glob(data_file)
  n_volumes = len(folders)
  
  volumes = np.zeros((n_volumes, *image_size, 1))
  labels = np.zeros((n_volumes, *image_size, 1))

  i = 0

  for filename in glob.glob(data_file):
    
    #print(filename)
    name = filename[-7:]
    #print(name)

    img_data = nib.load('/content/drive/My Drive/MISA/Normal Segmentations/data/{}/{}/{}_bias_corrected.nii.gz'.format(setName, name, name))
    img_data_temp = img_data.get_fdata()
    img_data_temp = img_data_temp.reshape((*image_size, 1))
    #print(img_data_temp.shape)
    volumes[i] = img_data_temp

    seg_data = nib.load('/content/drive/My Drive/MISA/Normal Segmentations/data/{}/{}/{}_seg.nii.gz'.format(setName, name, name))
    labels[i] = seg_data.get_fdata()
    
    print("Working on image {0}".format(name))
    i = i+1

  return (volumes, labels)

In [ ]:
(t_volumes, t_labels) = load_data_bias(IMAGE_SIZE, 'Training_Set')
(v_volumes, v_labels) = load_data_bias(IMAGE_SIZE, 'Validation_Set')

Working on image IBSR_05
Working on image IBSR_03
Working on image IBSR_08
Working on image IBSR_04
Working on image IBSR_01
Working on image IBSR_16
Working on image IBSR_18
Working on image IBSR_07
Working on image IBSR_09
Working on image IBSR_06
Working on image IBSR_14
Working on image IBSR_17
Working on image IBSR_12
Working on image IBSR_13


##Splitting the Dataset

In [ ]:
# Split the training data into training and validation
training_volumes = t_volumes[TRAINING_VOLUMES]
training_labels = t_labels[TRAINING_VOLUMES]

validation_volumes = t_volumes[VALIDATION_VOLUMES]
validation_labels = t_labels[VALIDATION_VOLUMES]

print(training_volumes.shape)
#print(training_labels.shape)

print(validation_volumes.shape)
#print(validation_labels.shape)

(9, 256, 128, 256, 1)
(1, 256, 128, 256, 1)


##Extracting Patches

In [ ]:
def extract_patches(x, patch_size, patch_stride) :
  return tf.image.extract_patches(
    x,
    sizes=[1, *patch_size, 1],
    strides=[1, *patch_stride, 1],
    rates=[1, 1, 1, 1],
    padding='SAME', name=None)

In [ ]:
def extract_useful_patches(
    volumes, labels,
    image_size=IMAGE_SIZE,
    patch_size=PATCH_SIZE,
    stride=PATCH_STRIDE,
    threshold=CONTENT_THRESHOLD,
    num_classes=N_CLASSES) :

  volumes = volumes.reshape([-1, image_size[1], image_size[2], 1])
  labels = labels.reshape([-1, image_size[1], image_size[2], 1])

  vol_patches = extract_patches(volumes, patch_size, stride).numpy()
  seg_patches = extract_patches(labels, patch_size, stride).numpy()

  vol_patches = vol_patches.reshape([-1, *patch_size, 1])
  seg_patches = seg_patches.reshape([-1, *patch_size, ])

  # this will get rid of the background and only take foreground
  foreground_mask = seg_patches != 0

  # we only keep the useful forground patches
  # threshold too small - takes even the useless patches
  # threshold too high - might leave out useful patches
  useful_patches = foreground_mask.sum(axis=(1, 2)) > threshold * np.prod(patch_size)

  vol_patches = vol_patches[useful_patches]
  seg_patches = seg_patches[useful_patches]

  seg_patches = tf.keras.utils.to_categorical(
    seg_patches, num_classes=N_CLASSES, dtype='float32')
  
  return (vol_patches, seg_patches)

In [ ]:
# extract patches from training set
(training_patches, training_patches_seg) = extract_useful_patches(training_volumes, training_labels)

# extract patches from validation set
(validation_patches, validation_patches_seg) = extract_useful_patches(validation_volumes, validation_labels)

print(training_patches.shape)

(11546, 32, 32, 1)


##Data Augmentation

In [ ]:
# Degree of Augmentation
deg     = 0.2

datagen = ImageDataGenerator(
        rotation_range=40, #40
        width_shift_range=deg,
        height_shift_range=deg,
        # rescale=1./255,
        shear_range=deg,
        zoom_range=deg,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest') #reflect, wrap, constant(black)

In [ ]:
train_generator = datagen.flow(training_patches, batch_size=int(training_patches.shape[0]/BATCH_SIZE), seed=1)
train_label_generator = datagen.flow(training_patches_seg, batch_size=int(training_patches.shape[0]/BATCH_SIZE), seed=1)

val_generator = datagen.flow(validation_patches, batch_size=int(validation_patches.shape[0]/BATCH_SIZE), seed=1)
val_label_generator = datagen.flow(validation_patches_seg, batch_size=int(validation_patches.shape[0]/BATCH_SIZE), seed=1)

In [ ]:
X_train = train_generator.next()
y_train = train_label_generator.next()

X_val = val_generator.next()
y_val = val_label_generator.next()

In [ ]:
print(training_patches.shape)
print(training_patches_seg.shape)
print("----------------")
print(validation_patches.shape)
print(validation_patches_seg.shape)

(11546, 32, 32, 1)
(11546, 32, 32, 4)
----------------
(1201, 32, 32, 1)
(1201, 32, 32, 4)


In [ ]:
full_train = np.concatenate((training_patches, X_train))
print(full_train.shape)
full_train_label = np.concatenate((training_patches_seg, y_train))
print(full_train_label.shape)

full_val = np.concatenate((validation_patches, X_val))
print(full_val.shape)
full_val_label = np.concatenate((validation_patches_seg, y_val))
print(full_val_label.shape)

(11726, 32, 32, 1)
(11726, 32, 32, 4)
(1219, 32, 32, 1)
(1219, 32, 32, 4)


##Train the Model

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=PATIENCE), # early stopping
    tf.keras.callbacks.ModelCheckpoint(filepath=MODEL_FNAME_PATTERN, save_best_only=True) # save the best based on validation
]

unet = get_unet()
unet.compile(optimizer=OPTIMISER, loss=LOSS)
unet.fit(
    x=full_train, 
    y=full_train_label,
    validation_data=(full_val, full_val_label),
    batch_size=BATCH_SIZE,
    epochs=N_EPOCHS,
    callbacks=my_callbacks,
    verbose=1)

Epoch 1/200
184/184 [==============================] - 27s 135ms/step - loss: 1.0389 - val_loss: 0.5890
Epoch 2/200
184/184 [==============================] - 24s 131ms/step - loss: 0.4410 - val_loss: 0.4919
Epoch 3/200
184/184 [==============================] - 24s 131ms/step - loss: 0.3838 - val_loss: 0.4056
Epoch 4/200
184/184 [==============================] - 24s 128ms/step - loss: 0.3393 - val_loss: 0.4225
Epoch 5/200
184/184 [==============================] - 24s 132ms/step - loss: 0.3046 - val_loss: 0.3648
Epoch 6/200
184/184 [==============================] - 24s 128ms/step - loss: 0.2799 - val_loss: 0.3749
Epoch 7/200
184/184 [==============================] - 24s 130ms/step - loss: 0.2612 - val_loss: 0.3762
Epoch 8/200
184/184 [==============================] - 24s 128ms/step - loss: 0.2467 - val_loss: 0.4251
Epoch 9/200
184/184 [==============================] - 24s 128ms/step - loss: 0.2360 - val_loss: 0.3782
Epoch 10/200
184/184 [==============================] - 24s 129m

##Load the best model

In [ ]:
unet = get_unet(
    img_size=(IMAGE_SIZE[1], IMAGE_SIZE[2]),
    n_classes=N_CLASSES,
    n_input_channels=N_INPUT_CHANNELS)
unet.compile(optimizer=OPTIMISER, loss=LOSS)
unet.load_weights('model.h5')

##Prepare test data using the validation volumes

In [ ]:
def prepare_val_data(the_volumes, the_labels):
  testing_volumes_processed = the_volumes.reshape([-1, IMAGE_SIZE[1], IMAGE_SIZE[2], 1])
  testing_labels_processed = the_labels.reshape([-1, IMAGE_SIZE[1], IMAGE_SIZE[2], 1])

  testing_labels_processed = tf.keras.utils.to_categorical(testing_labels_processed, num_classes=4, dtype='float32')

  #print(testing_volumes_processed.shape)
  #print(testing_labels_processed.shape)

  return (testing_volumes_processed, testing_labels_processed)

###Predict labels for test data

In [ ]:
def pred_val_data(testing_volumes_processed)  :
  # creates probability map of each label for all volumes
  prediction = unet.predict(x=testing_volumes_processed)

  prediction = np.argmax(prediction, axis=3)

  #plt.axis('off')
  #plt.imshow(prediction[:, :, 150])

  return prediction

In [ ]:
"""
print(prediction.shape)
print(testing_labels_processed.shape)
print(testing_volumes_T1_processed.shape)
"""

'\nprint(prediction.shape)\nprint(testing_labels_processed.shape)\nprint(testing_volumes_T1_processed.shape)\n'

##Computing Dice, AVD and HD (Final)



In [ ]:
def compute_hausdorff_distance(in1, in2, label = 'all'):
    in1=in1.squeeze()
    in2=in2.squeeze()
    hausdorff_distance_filter = sitk.HausdorffDistanceImageFilter()
    if label == 'all':
        # Hausdorff distance
        hausdorff_distance_filter.Execute(in1, in2)
    else:
        in1_array  = in1 #sitk.GetArrayFromImage(in1)
        in1_array = (in1_array == label) *1 
        in1_array = in1_array.astype('uint16')  
        img1 = sitk.GetImageFromArray(in1_array)
        
        in2_array  = in2 #sitk.GetArrayFromImage(in2)
        in2_array = (in2_array == label) *1 
        in2_array = in2_array.astype('uint16')  
        img2 = sitk.GetImageFromArray(in2_array)
        # Hausdorff distance
        hausdorff_distance_filter.Execute(img1, img2)
    return hausdorff_distance_filter.GetHausdorffDistance()

def compute_dice_coefficient(in1, in2, label  = 'all'):
    in1=in1.squeeze()
    in2=in2.squeeze()
    if label=='all': 
        return 2 * np.sum( (in1>0) &  (in2>0) & (in1 == in2)) / (np.sum(in1 > 0) + np.sum(in2 > 0))
    else:
        return 2 * np.sum((in1 == label) & (in2 == label)) / (np.sum(in1 == label) + np.sum(in2 == label))

def compute_volumentric_difference(in1, in2, label  = 'all'):
    in1=in1.squeeze()
    in2=in2.squeeze()
    if label  == 'all':
      #  vol_dif  = np.sum((in1 != in2) & (in1 !=0) & (in2 !=0))
        return np.sum((in1 != in2)) / ((np.sum(in1 > 0) + np.sum(in2 > 0)))
    else:
        in1  = (in1 == label) * 1
        in2  = (in2 == label) * 1
        return np.sum((in1 != in2)) / ((np.sum(in1 > 0) + np.sum(in2 > 0)))

In [ ]:
for cl in range(0,4,1): 
  overallDSC = np.zeros(N_CLASSES)
  overall_Hausdorff = np.zeros(N_CLASSES)
  overall_vol = np.zeros(N_CLASSES)

  for i in range(0,validation_volumes.shape[0], 1):
      
      testing_volumes_processed, testing_labels_processed = prepare_val_data(v_volumes[i], v_labels[i])
      prediction = pred_val_data(testing_volumes_processed)
      
      #cl = 3

      cur_DSC = compute_dice_coefficient(prediction, v_labels[i], label=cl)
      overallDSC = overallDSC + cur_DSC

      cur_Hausdorff = compute_hausdorff_distance(prediction, v_labels[i], label=cl) 
      overall_Hausdorff = overall_Hausdorff + cur_Hausdorff

      cur_vol = compute_volumentric_difference(prediction, v_labels[i], label=cl)
      overall_vol = overall_vol + cur_vol
      
      #print(prediction.shape)
      #print(v_labels[i].shape)
      
  #print(overall_Hausdorff)
  overallDSC = overallDSC/validation_volumes.shape[0]
  overall_Hausdorff = overall_Hausdorff/validation_volumes.shape[0]
  overall_vol = overall_vol/validation_volumes.shape[0]

  # for i in range(0,cl,1):
  #print("Class {} - Dice Coefficient = {:.4f}".format(cl, overallDSC[i]))
  #print("Class {} - HD = {:.4f}".format(cl, overall_Hausdorff[i]))
  #print("Class {} - AVD = {:.4f}".format(cl, overall_vol[i]))
  print("Class {}".format(cl))
  print("\tDice Coefficient = {:.4f}".format(overallDSC[i]))
  # print("\tHD = {:.4f}".format(overall_Hausdorff[i]))
  # print("\tAVD = {:.4f}".format(overall_vol[i]))

Class 0
	Dice Coefficient = 0.9913
Class 1
	Dice Coefficient = 0.7088
Class 2
	Dice Coefficient = 0.7517
Class 3
	Dice Coefficient = 0.4736


In [ ]:
# 2DUNet
# batch size = 32, patient = 5, dropout=0.15, epoch = 50
"""
Class 0 - Dice Coefficient 0.9976
Class 1 - Dice Coefficient 0.8288
Class 2 - Dice Coefficient 0.9186
Class 3 - Dice Coefficient 0.8765
"""

# batch size = 40, patient = 5, dropout=0.15, epoch = 50
"""
Class 0 - Dice Coefficient 0.9977
Class 1 - Dice Coefficient 0.8261
Class 2 - Dice Coefficient 0.9202
Class 3 - Dice Coefficient 0.8790
"""

# batch size = 50, patient = 20, dropout=0.15, epoch = 200
"""
Class 0 - Dice Coefficient 0.9977
Class 1 - Dice Coefficient 0.8261
Class 2 - Dice Coefficient 0.9202
Class 3 - Dice Coefficient 0.8790
"""

# batch size = 64, patient = 20, dropout=0.40, epoch = 200
"""
Class 0 - Dice Coefficient 0.9975
Class 1 - Dice Coefficient 0.8342
Class 2 - Dice Coefficient 0.9209
Class 3 - Dice Coefficient 0.8825
"""

'\nClass 0 - Dice Coefficient 0.9975\nClass 1 - Dice Coefficient 0.8342\nClass 2 - Dice Coefficient 0.9209\nClass 3 - Dice Coefficient 0.8825\n'